In [1]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from pathlib import Path

In [2]:
#Selecionando as Variáveis
ano_dt_base = 2020
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

Colunas que vai precisar pro cálculo:

LOTAÇÃO	MATRICULA	GRUPO	CRITERIO DE APOSENTADORIA	SEXO	DTNASC	DTADM	DATACARGO	SALPART	CARREIRA	TSANT	CONJUGE	DTNASCONJ	DTNASDEPINV	DTNASDEPVAL

In [28]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')


In [29]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
statis_ativos['IDADE_ADM'] =  bd_ativos['DT_ING_ENTE'].dt.strftime('%Y').astype(int) - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#---------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos['DT_NASC_CONJUGE'].mask(bd_ativos['DT_NASC_CONJUGE'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_CONJUGE'] = ano_dt_base - statis_ativos['IDADE_CONJUGE'].dt.strftime('%Y').astype(int) #Calculando as idades
statis_ativos.loc[(bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 1) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 3) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 4) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 5), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#statistics_ativos['IDADE_CONJUGE'] =  ano_dt_base - bd_ativos['DT_NASC_CONJUGE'].dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Cônjuge
#statistics_ativos['IDADE'] = ano_dt_base - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int)
#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')

statis_ativos.head(15)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE
0,40,20,NaN
1,45,42,48.0
2,48,27,NaN
3,31,24,NaN
4,42,35,NaN
5,51,29,NaN
6,40,22,53.0
7,40,34,NaN
8,45,42,NaN
9,41,31,NaN


In [30]:
statis_ativos['IDADE_CONJUGE'].mean()


163.66438356164383

In [13]:
bd_ativos['CO_EST_CIVIL_SERVIDOR'].head(15)

0     1
1     2
2     1
3     1
4     1
5     1
6     2
7     5
8     5
9     1
10    1
11    6
12    1
13    1
14    1
Name: CO_EST_CIVIL_SERVIDOR, dtype: int64

In [7]:
statis_ativos.head(15)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE
0,40,20,NaN
1,45,42,220.0
2,48,27,NaN
3,31,24,NaN
4,42,35,NaN
5,51,29,NaN
6,40,22,220.0
7,40,34,220.0
8,45,42,220.0
9,41,31,NaN


In [ ]:
#statistics_ativos['IDADE_CONJUGE'] = np.where(bd_ativos['DT_NASC_CONJUGE'] == '', "False", "True")
#statistics_ativos['IDADE_CONJUGE'] = bd_ativos['DT_NASC_CONJUGE'].apply(lambda x: 'True' if x pd.isna(bd_ativos['DT_NASC_CONJUGE']) else 'False')
#statistics_ativos['IDADE_CONJUGE'].apply(lambda x: 'True' if x.bd_ativos['NU_TEMPO_RGPS'] > 5000 else 'False')
#statistics_ativos['IDADE_CONJUGE'] = np.where(pd.isna(bd_ativos['DT_NASC_CONJUGE']), "", "10")
#statistics_ativos['IDADE_CONJUGE'].head(15)

for i in range(127):
    if pd.notnull(bd_ativos['DT_NASC_CONJUGE']).loc[i] == 'True':
        statistics_ativos['IDADE_CONJUGE'].loc[i] = ano_dt_base - bd_ativos['DT_NASC_CONJUGE'].dt.strftime('%Y').astype(int).loc[i]
    else:
        statistics_ativos['IDADE_CONJUGE'].loc[i] = ''

statistics_ativos['IDADE_CONJUGE'].head(15)

In [ ]:
#statistics_ativos['IDADE_CONJUGE'] = np.where(pd.isna(bd_ativos['DT_NASC_CONJUGE']), "", "10")
statistics_ativos['IDADE_CONJUGE'] = pd.notnull(bd_ativos['DT_NASC_CONJUGE'])
statistics_ativos['IDADE_CONJUGE'].head(15)
#statistics_ativos['IDADE_CONJUGE'].loc[9]

In [ ]:
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
#bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].astype(float)
bd_ativos['TESTE'] = bd_ativos['VL_BASE_CALCULO'] + 1000
bd_ativos[['TESTE', 'VL_BASE_CALCULO']]